In [24]:
import json
import pandas as pd
import requests

In [ ]:
path = "../../data/SQUAD_v2.0/train.json"
with open(path) as f:
    data = json.load(f)

In [7]:
data.keys()

dict_keys(['version', 'data'])

In [26]:
def find_titles_by_text(query, limit = 10):
        url = "https://en.wikipedia.org/w/api.php"
        params = {
            "srsearch": query,
            "srlimit": limit,
            "srwhat" : "text",
            "action": "query",
            "format": "json",
            "list": "search",
        }
        res = requests.get(url, params)
        return res.json()

In [80]:
def build_page_dataset(data, max_questions = 50):
    res = pd.DataFrame(columns = ["question", "label"])
    for data_subset in data["data"]:
        print("Extracting: {}".format(data_subset["title"]))
        n_questions = 0
        for paragraph in data_subset["paragraphs"]:
            if n_questions > max_questions:
                    break
            context = paragraph["context"]
            label = find_titles_by_text(context[:300], limit = 1)
            if len(label["query"]["search"]) > 0:
                label = label["query"]["search"][0]["title"]
            else:
                continue
            for question in paragraph["qas"]:
                if n_questions > max_questions:
                    break
                if not question["is_impossible"]:
                    res.loc[len(res)] = {"question": question["question"], "label": label}
                    n_questions += 1
    return res

In [82]:
page_dataset = build_page_dataset(data, max_questions = 10)

Extracting: Beyoncé
Extracting: Frédéric_Chopin
Extracting: Sino-Tibetan_relations_during_the_Ming_dynasty
Extracting: IPod
Extracting: The_Legend_of_Zelda:_Twilight_Princess
Extracting: Spectre_(2015_film)
Extracting: 2008_Sichuan_earthquake
Extracting: New_York_City
Extracting: To_Kill_a_Mockingbird
Extracting: Solar_energy
Extracting: Kanye_West
Extracting: Buddhism
Extracting: American_Idol
Extracting: Dog
Extracting: 2008_Summer_Olympics_torch_relay
Extracting: Genome
Extracting: Comprehensive_school
Extracting: Republic_of_the_Congo
Extracting: Prime_minister
Extracting: Institute_of_technology
Extracting: Wayback_Machine
Extracting: Dutch_Republic
Extracting: Symbiosis
Extracting: Canadian_Armed_Forces
Extracting: Cardinal_(Catholicism)
Extracting: Iranian_languages
Extracting: Lighting
Extracting: Separation_of_powers_under_the_United_States_Constitution
Extracting: Architecture
Extracting: Human_Development_Index
Extracting: Southern_Europe
Extracting: BBC_Television
Extractin

Extracting: George_VI
Extracting: Federalism
Extracting: Annelid
Extracting: God
Extracting: War_on_Terror
Extracting: Labour_Party_(UK)
Extracting: Estonia
Extracting: Alaska
Extracting: Karl_Popper
Extracting: Mandolin
Extracting: Insect
Extracting: Race_(human_categorization)
Extracting: Paris
Extracting: Apollo
Extracting: United_States_presidential_election,_2004
Extracting: Liberal_Party_of_Australia
Extracting: Samurai
Extracting: Software_testing
Extracting: States_of_Germany
Extracting: Glass
Extracting: Planck_constant
Extracting: Renewable_energy_commercialization
Extracting: Palermo
Extracting: Green
Extracting: Zinc
Extracting: Neoclassical_architecture
Extracting: Serbo-Croatian
Extracting: CBC_Television
Extracting: Appalachian_Mountains
Extracting: IBM
Extracting: Energy
Extracting: East_Prussia
Extracting: Ottoman_Empire
Extracting: Philosophy_of_space_and_time
Extracting: Neolithic
Extracting: Friedrich_Hayek
Extracting: Diarrhea
Extracting: Madrasa
Extracting: Miami


In [87]:
out_path = "../../data/page_dataset/page_dataset_10.csv"
page_dataset.to_csv(out_path, index = False)